Cifrado Asimétrico — ECC

In [1]:
!pip install cryptography

In [4]:
import os, time, base64
import pandas as pd
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

def generar_archivo(nombre, num_palabras):
    """Genera un archivo con num_palabras iguales."""
    with open(nombre, "w") as f:
        f.write(("hola " * num_palabras).strip())

def leer_archivo(nombre):
    with open(nombre, "r") as f:
        return f.read()

def generar_clave_ecdh_sin_hash():
    """Genera una clave simétrica usando solo ECDH (sin HKDF, sin hash)"""
    private_key = ec.generate_private_key(ec.SECP256R1())
    peer_private = ec.generate_private_key(ec.SECP256R1())

    # ECDH → clave compartida cruda
    shared_key = private_key.exchange(ec.ECDH(), peer_private.public_key())

    # AES-GCM necesita 32 bytes = 256 bits
    derived_key = shared_key[:32]

    return derived_key

def cifrar(derived_key, mensaje):
    aesgcm = AESGCM(derived_key)
    nonce = os.urandom(12)
    ciphertext = aesgcm.encrypt(nonce, mensaje.encode(), None)
    return nonce, ciphertext

def descifrar(derived_key, nonce, ciphertext):
    aesgcm = AESGCM(derived_key)
    return aesgcm.decrypt(nonce, ciphertext, None).decode()

tamaños = [10, 100, 1000, 10000, 100000, 1000000, 10000000]
resultados = []

for palabras in tamaños:
    print(f"\n Procesando archivo con {palabras} palabras...")

    nombre_archivo = f"text_{palabras}.txt"

    # ----------------- ETAPA 1: Leer archivo -----------------
    t1 = time.time()
    generar_archivo(nombre_archivo, palabras)
    texto = leer_archivo(nombre_archivo)
    t2 = time.time()

    # ----------------- ETAPA 2: Generar clave ----------------
    t3 = time.time()
    derived_key = generar_clave_ecdh_sin_hash()
    t4 = time.time()

    # ----------------- ETAPA 3: Cifrar -----------------------
    t5 = time.time()
    nonce, ciphertext = cifrar(derived_key, texto)
    t6 = time.time()

    # ----------------- ETAPA 4: Descifrar --------------------
    t7 = time.time()
    text_plain = descifrar(derived_key, nonce, ciphertext)
    t8 = time.time()

    resultados.append([
        "ECDH(sin-hash)+AES-GCM",
        palabras,
        len(texto),
        len(ciphertext),
        t2 - t1,
        t4 - t3,
        t6 - t5,
        t8 - t7,
        t8 - t1
    ])

df = pd.DataFrame(resultados, columns=[
    "Algoritmo",
    "#Palabras",
    "#Caracteres_Entrada",
    "#Caracteres_Salida",
    "T-E1 (Leer)",
    "T-E2 (Clave)",
    "T-E3 (Cifrar)",
    "T-E4 (Descifrar)",
    "T-Total",
])

print("\n=== TABLA RESUMEN ===")
print(df)

df.to_csv("benchmark_results.csv", index=False)
print("\nCSV generado: benchmark_results.csv")



 Procesando archivo con 10 palabras...

 Procesando archivo con 100 palabras...

 Procesando archivo con 1000 palabras...

 Procesando archivo con 10000 palabras...

 Procesando archivo con 100000 palabras...

 Procesando archivo con 1000000 palabras...

 Procesando archivo con 10000000 palabras...

=== TABLA RESUMEN ===
                Algoritmo  #Palabras  #Caracteres_Entrada  #Caracteres_Salida  \
0  ECDH(sin-hash)+AES-GCM         10                   49                  65   
1  ECDH(sin-hash)+AES-GCM        100                  499                 515   
2  ECDH(sin-hash)+AES-GCM       1000                 4999                5015   
3  ECDH(sin-hash)+AES-GCM      10000                49999               50015   
4  ECDH(sin-hash)+AES-GCM     100000               499999              500015   
5  ECDH(sin-hash)+AES-GCM    1000000              4999999             5000015   
6  ECDH(sin-hash)+AES-GCM   10000000             49999999            50000015   

   T-E1 (Leer)  T-E2 (Clave